In [1]:
import os 
import xlsxwriter
import pandas as pd 
from utils import *
import plotly.graph_objects as go

In [2]:
est_brasilia = carregar_dados_csv(
    caminho_arquivo('dados_83377_D_1961-09-11_2024-03-14.csv'),
                                    delimiter=';', skiprows=10)


est_brasilia = est_brasilia[est_brasilia['Data Medicao'] < '2024-01-01']

Verificar a existência de valores vazios

In [3]:
contar_valores_nodata(est_brasilia, 'PRECIPITACAO TOTAL, DIARIO(mm)')

'Completo: 22757 | Buracos (NAN): 205  | Sem buracos: 22552'

In [4]:
contar_valores_nodata(est_brasilia, 'UMIDADE RELATIVA DO AR, MEDIA DIARIA(%)')

'Completo: 22757 | Buracos (NAN): 259  | Sem buracos: 22498'

In [5]:
contar_valores_nodata(est_brasilia,'TEMPERATURA MAXIMA, DIARIA(°C)' )

'Completo: 22757 | Buracos (NAN): 190  | Sem buracos: 22567'

In [6]:
contar_valores_nodata(est_brasilia, 'TEMPERATURA MINIMA, DIARIA(°C)')

'Completo: 22757 | Buracos (NAN): 166  | Sem buracos: 22591'

Aplicar regras de negócio:
- Regra 1: Remover valores vazios
- Regra 2: Inserir mes e ano
    

In [7]:
est_brasilia_filtro_precipitacao = filtrar_dados_notNaN(est_brasilia, 'PRECIPITACAO TOTAL, DIARIO(mm)')
est_brasilia_filtro_umidade = filtrar_dados_notNaN(est_brasilia, 'UMIDADE RELATIVA DO AR, MEDIA DIARIA(%)')
est_brasilia_filtro_min = filtrar_dados_notNaN(est_brasilia, 'TEMPERATURA MINIMA, DIARIA(°C)')
est_brasilia_filtro_max = filtrar_dados_notNaN(est_brasilia, 'TEMPERATURA MAXIMA, DIARIA(°C)')


In [8]:
est_brasilia_filtro_precipitacao = inserir_mes_ano(est_brasilia_filtro_precipitacao, 'Data Medicao')
est_brasilia_filtro_umidade = inserir_mes_ano(est_brasilia_filtro_umidade, 'Data Medicao')
est_brasilia_filtro_min = inserir_mes_ano(est_brasilia_filtro_min, 'Data Medicao')
est_brasilia_filtro_max = inserir_mes_ano(est_brasilia_filtro_max, 'Data Medicao')

In [9]:
est_brasilia_filtro_precipitacao = inserir_numero_mes(est_brasilia_filtro_precipitacao)
est_brasilia_filtro_umidade = inserir_numero_mes(est_brasilia_filtro_umidade)
est_brasilia_filtro_min = inserir_numero_mes(est_brasilia_filtro_min)
est_brasilia_filtro_max = inserir_numero_mes(est_brasilia_filtro_max)


Filtrar as colunas de interesse

In [10]:
# 

est_brasilia_select_precipitacao = est_brasilia_filtro_precipitacao[['Data Medicao','NUM_MÊS','MÊS','ANO','PRECIPITACAO TOTAL, DIARIO(mm)']]
est_brasilia_select_umidade = est_brasilia_filtro_umidade[['Data Medicao','NUM_MÊS','MÊS','ANO','UMIDADE RELATIVA DO AR, MEDIA DIARIA(%)']]
est_brasilia_select_max = est_brasilia_filtro_max[['Data Medicao','NUM_MÊS','MÊS','ANO','TEMPERATURA MAXIMA, DIARIA(°C)']]
est_brasilia_select_min = est_brasilia_filtro_min[['Data Medicao','NUM_MÊS','MÊS','ANO','TEMPERATURA MINIMA, DIARIA(°C)']]


Aplicar regra de negócio:
- Regra 3: Verificar anos com registros em mais de 360 dias
- Regra 4: Verificar quantos anos estão completos e quantos estão incompletos

In [11]:
completos_brasilia_precipitacao, incompletos_brasilia_precipitacao = listar_anos_descartados(est_brasilia_select_precipitacao, 'ANO')
print(f'Anos Completos: {len(completos_brasilia_precipitacao)}\nAnos Incompletos: {len(incompletos_brasilia_precipitacao)}')

Anos Completos: 58
Anos Incompletos: 5


In [12]:
completos_brasilia_umidade, incompletos_brasilia_umidade = listar_anos_descartados(est_brasilia_select_umidade, 'ANO')
print(f'Anos Completos: {len(completos_brasilia_umidade)}\nAnos Incompletos: {len(incompletos_brasilia_umidade)}')

Anos Completos: 58
Anos Incompletos: 5


In [13]:
completos_brasilia_min, incompletos_brasilia_min = listar_anos_descartados(est_brasilia_select_min, 'ANO')
print(f'Anos Completos: {len(completos_brasilia_min)}\nAnos Incompletos: {len(incompletos_brasilia_min)}')

Anos Completos: 59
Anos Incompletos: 4


In [14]:
completos_brasilia_max, incompletos_brasilia_max = listar_anos_descartados(est_brasilia_select_max, 'ANO')
print(f'Anos Completos: {len(completos_brasilia_max)}\nAnos Incompletos: {len(incompletos_brasilia_max)}')

Anos Completos: 58
Anos Incompletos: 4


In [15]:
# Utilizando somente os anos completos na elaboração dos gráficos

est_brasilia_select_precipitacao = est_brasilia_select_precipitacao[est_brasilia_select_precipitacao['ANO'].isin(completos_brasilia_precipitacao)]
est_brasilia_select_umidade = est_brasilia_select_umidade[est_brasilia_select_umidade['ANO'].isin(completos_brasilia_umidade)]
est_brasilia_select_max = est_brasilia_select_max[est_brasilia_select_max['ANO'].isin(completos_brasilia_max)]
est_brasilia_select_min = est_brasilia_select_min[est_brasilia_select_min['ANO'].isin(completos_brasilia_min)]

### CALCULAR
- PRECIPITAÇÃO MENSAL = SOMA DA PRECIPITAÇÃO DE CADA DIA DO MÊS PARA CADA ANO
- MÉDIA ANUAL MENSAL = MÉDIA DA PRECIPITAÇÃO DE CADA MÊS PARA TODOS OS ANOS

In [16]:
# Calculo somente dos anos completos
# est_brasilia_precipitacao_acumulado_mes = calcular_precipitacao_acumulada_mes(est_brasilia_select_precipitacao[est_brasilia_select_precipitacao['ANO'].isin(completos_brasilia_precipitacao)], ['ANO','NUM_MÊS','MÊS'], 'PRECIPITACAO TOTAL, DIARIO(mm)')

est_brasilia_precipitacao_acumulado_mes = calcular_precipitacao_acumulada_mes(est_brasilia_select_precipitacao, ['ANO','NUM_MÊS','MÊS'], 'PRECIPITACAO TOTAL, DIARIO(mm)')

In [17]:
est_brasilia_precipitacao_media_ano = calcular_media_ano_completo(est_brasilia_precipitacao_acumulado_mes, ['NUM_MÊS','MÊS'], 'PRECIPITACAO TOTAL, DIARIO(mm)')

In [18]:
# Reorganizar os dados para que o gráfico comece pelo mês 9

jan_ago = est_brasilia_precipitacao_media_ano[est_brasilia_precipitacao_media_ano['NUM_MÊS'] < 9]
set_dez = est_brasilia_precipitacao_media_ano[est_brasilia_precipitacao_media_ano['NUM_MÊS'] >= 9]

est_brasilia_precipitacao_media_ano_sorted = pd.concat([set_dez, jan_ago])

# Criar o gráfico
fig = go.Figure()

# Adicionar o trace com os dados ordenados
fig.add_trace(go.Scatter(x=est_brasilia_precipitacao_media_ano_sorted['MÊS'], 
                         y=round(est_brasilia_precipitacao_media_ano_sorted['PRECIPITACAO TOTAL, DIARIO(mm)'], 2), 
                         mode='lines+markers+text', 
                         text = round(est_brasilia_precipitacao_media_ano_sorted['PRECIPITACAO TOTAL, DIARIO(mm)'], 0),
                         textposition='top center',
                         name='Estação Convencional Inmet (1961 - 2023)'))

# Atualizar o layout para garantir que o eixo x comece pelo mês 9
fig.update_layout(
    xaxis=dict(
        tickvals=['September', 'October', 'November', 'December', 'January', 'February',
                  'March', 'April', 'May', 'June', 'July', 'August'],  # Ordenação do eixo X começando pelo mês 9
        ticktext=['S', 'O', 'N', 'D', 'J/E', 'F', 'M', 'A', 'M', 'J', 'J', 'A'],  # Abreviações dos meses
        # title="Mês",
        # categoryorder="array",
        # categoryarray=[9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8]  # Definindo a ordem do eixo
    ),
    yaxis= dict(
        
        range=[0, 300],  # Definindo o intervalo do eixo Y
        title="Média de Chuva (mm)"
        ),  # Título do eixo Y
    title="Média histórica de chuva mensal no Distrito Federal (1961 - 2023)", # Título do gráfico
    
    # Adicionar a anotação com a fonte dos dados
    annotations = [
        dict(
            x=0, y=-0.2,  # Posição da fonte no layout (ajuste conforme necessário)
            xref="paper", yref="paper",
            text="Fonte: INMET, 2024",  # Texto da fonte
            showarrow=False,
            font=dict(size=10, color="gray")
        )
    ]
)


In [19]:
# Exportar o df para um arquivo csv

# est_brasilia_precipitacao_media_ano_sorted.rename(
#     columns={'NUM_MÊS':'num_mes',
#                 'MÊS':'nm_mes',
#                 'PRECIPITACAO TOTAL, DIARIO(mm)':'precipitacao_mensal_media_mm'
#              }, inplace=True)

# est_brasilia_precipitacao_media_ano_sorted['precipitacao_mensal_media_mm'] = round(est_brasilia_precipitacao_media_ano_sorted['precipitacao_mensal_media_mm'], 2)

# if not os.path.exists(caminho_arquivo('resultado')):
#     os.makedirs(caminho_arquivo('resultado'))
# est_brasilia_precipitacao_media_ano_sorted.to_csv(caminho_arquivo('resultado/est_brasilia_precipitacao_media_ano_sorted.csv'), index=False)

#### CALCULAR
- UMIDADE MENSAL = MÉDIA DA UMIDADE DE CADA DIA DO MÊS PARA CADA ANO
- UMIDADE ANUAL = MÉDIA DA UMIDADE DE CADA MÊS PARA TODOS OS ANOS

In [20]:
est_brasilia_umidade_media_mes = calcular_media_mes(est_brasilia_select_umidade, ['ANO','NUM_MÊS','MÊS'], 'UMIDADE RELATIVA DO AR, MEDIA DIARIA(%)')

In [21]:
est_brasilia_umidade_media_ano = calcular_media_ano_completo(est_brasilia_umidade_media_mes, ['NUM_MÊS','MÊS'], 'UMIDADE RELATIVA DO AR, MEDIA DIARIA(%)')

In [22]:
# Criar o gráfico
fig = go.Figure()

# Adicionar o trace com os dados ordenados
fig.add_trace(go.Scatter(x=est_brasilia_umidade_media_ano['MÊS'], 
                         y=round(est_brasilia_umidade_media_ano['UMIDADE RELATIVA DO AR, MEDIA DIARIA(%)'], 2), 
                         mode='lines+markers+text', 
                         text = round(est_brasilia_umidade_media_ano['UMIDADE RELATIVA DO AR, MEDIA DIARIA(%)'], 0),
                         textposition='top center',
                         name='Estação Convencional Inmet (1961 - 2023)'))

# Atualizar o layout para garantir que o eixo x comece pelo mês 9
fig.update_layout(
    xaxis=dict(
        tickvals=['September', 'October', 'November', 'December', 'January', 'February',
                  'March', 'April', 'May', 'June', 'July', 'August'],  # Ordenação do eixo X começando pelo mês 9
        ticktext=['S', 'O', 'N', 'D', 'J/E', 'F', 'M', 'A', 'M', 'J', 'J', 'A'],  # Abreviações dos meses
        # title="Mês"
    ),
    yaxis = dict(
        title = 'Média da umidade (%)',
        range = [0, 85]
    ),
    title="Média histórica de umidade relativa do ar no Distrito Federal (1961 - 2023)", # Título do gráfico
    # Adicionar a anotação com a fonte dos dados
    annotations = [
        dict(
            x=0, y=-0.2,  # Posição da fonte no layout (ajuste conforme necessário)
            xref="paper", yref="paper",
            text="Fonte: INMET, 2024",  # Texto da fonte
            showarrow=False,
            font=dict(size=10, color="gray")
        )
    ]
)


In [23]:
# Exportar o df para um arquivo csv

# est_brasilia_umidade_media_ano.rename(
#     columns={'NUM_MÊS':'num_mes',
#                 'MÊS':'nm_mes',
#                 'UMIDADE RELATIVA DO AR, MEDIA DIARIA(%)':'umidade_relativa_ar_media_%'
#              }, inplace=True)

# est_brasilia_umidade_media_ano['umidade_relativa_ar_media_%'] = round(est_brasilia_umidade_media_ano['umidade_relativa_ar_media_%'], 2)

# if not os.path.exists(caminho_arquivo('resultado')):
#     os.makedirs(caminho_arquivo('resultado'))
# est_brasilia_umidade_media_ano.to_csv(caminho_arquivo('resultado/est_brasilia_umidade_media_ano.csv'), index=False)

#### CALCULAR
- TEMPERATURA MAX MENSAL = MÉDIA DA TEMPERATURA MAX DE CADA DIA DO MÊS PARA CADA ANO
- TEMPERATURA MAX ANUAL = MÉDIA DA TEMPERATURA MAX DE CADA MÊS PARA TODOS OS ANOS

In [24]:
est_brasilia_max_media_mes = calcular_media_mes(est_brasilia_select_max, ['ANO','NUM_MÊS','MÊS'], 'TEMPERATURA MAXIMA, DIARIA(°C)')

In [25]:
est_brasilia_max_media_ano = calcular_media_ano_completo(est_brasilia_max_media_mes, ['NUM_MÊS','MÊS'], 'TEMPERATURA MAXIMA, DIARIA(°C)')

In [26]:
# Criar o gráfico
fig = go.Figure()

# Adicionar o trace com os dados ordenados
fig.add_trace(go.Scatter(x=est_brasilia_max_media_ano['MÊS'], 
                         y=round(est_brasilia_max_media_ano['TEMPERATURA MAXIMA, DIARIA(°C)'], 2), 
                         mode='lines+markers+text', 
                         text = round(est_brasilia_max_media_ano['TEMPERATURA MAXIMA, DIARIA(°C)'], 0),
                         textposition='top center',
                         name='Estação Convencional Inmet (1961 - 2023)'))

# Atualizar o layout para garantir que o eixo x comece pelo mês 9
fig.update_layout(
    xaxis=dict(
        tickvals=['September', 'October', 'November', 'December', 'January', 'February',
                  'March', 'April', 'May', 'June', 'July', 'August'],  # Ordenação do eixo X começando pelo mês 9
        ticktext=['S', 'O', 'N', 'D', 'J/E', 'F', 'M', 'A', 'M', 'J', 'J', 'A'],  # Abreviações dos meses
        # title="Mês"
    ),
    yaxis = dict(
        title = 'Temperatura Máxima Média (°C)',
        range = [0, 32],
        tickvals = list(range(0,33, 5)),
    ),
    title="Média histórica de temperatura máxima no Distrito Federal (1961 - 2023)", # Título do gráfico
    # Adicionar a anotação com a fonte dos dados
    annotations = [
        dict(
            x=0, y=-0.2,  # Posição da fonte no layout (ajuste conforme necessário)
            xref="paper", yref="paper",
            text="Fonte: INMET, 2024",  # Texto da fonte
            showarrow=False,
            font=dict(size=10, color="gray")
        )
    ]
)


In [27]:
# Exportar o df para um arquivo csv

# est_brasilia_max_media_ano.rename(
#     columns={'NUM_MÊS':'num_mes',
#                 'MÊS':'nm_mes',
#                 'TEMPERATURA MAXIMA, DIARIA(°C)':'temperatura_max_media_°C'
#              }, inplace=True)

# est_brasilia_max_media_ano['temperatura_max_media_°C'] = round(est_brasilia_max_media_ano['temperatura_max_media_°C'], 2)

# if not os.path.exists(caminho_arquivo('resultado')):
#     os.makedirs(caminho_arquivo('resultado'))
# est_brasilia_max_media_ano.to_csv(caminho_arquivo('resultado/est_brasilia_max_media_ano.csv'), index=False)

#### CALCULAR
- TEMPERATURA MIN MENSAL = MÉDIA DA TEMPERATURA MIN DE CADA DIA DO MÊS PARA CADA ANO
- TEMPERATURA MIN ANUAL = MÉDIA DA TEMPERATURA MIN DE CADA MÊS PARA TODOS OS ANOS

In [28]:
est_brasilia_min_media_mes = calcular_media_mes(est_brasilia_select_min, ['ANO','NUM_MÊS','MÊS'], 'TEMPERATURA MINIMA, DIARIA(°C)')

In [29]:
est_brasilia_min_media_ano = calcular_media_ano_completo(est_brasilia_min_media_mes, ['NUM_MÊS','MÊS'], 'TEMPERATURA MINIMA, DIARIA(°C)')

In [30]:
# Criar o gráfico
fig = go.Figure()

# Adicionar o trace com os dados ordenados
fig.add_trace(go.Scatter(x=est_brasilia_min_media_ano['MÊS'], 
                         y=round(est_brasilia_min_media_ano['TEMPERATURA MINIMA, DIARIA(°C)'], 2), 
                         mode='lines+markers+text', 
                         text = round(est_brasilia_min_media_ano['TEMPERATURA MINIMA, DIARIA(°C)'], 0),
                         textposition='top center',
                         name='Estação Convencional Inmet (1961 - 2023)'))

# Atualizar o layout para garantir que o eixo x comece pelo mês 9
fig.update_layout(
    xaxis=dict(
        tickvals=['September', 'October', 'November', 'December', 'January', 'February',
                  'March', 'April', 'May', 'June', 'July', 'August'],  # Ordenação do eixo X começando pelo mês 9
        ticktext=['S', 'O', 'N', 'D', 'J/E', 'F', 'M', 'A', 'M', 'J', 'J', 'A'],  # Abreviações dos meses
        # title="Mês"
    ),
    yaxis = dict(
        title = 'Temperatura Mínima Média (°C)',
        range = [0, 20],
        tickvals = list(range(0,21, 2)),
    ),
    title="Média histórica de temperatura mínima no Distrito Federal (1961 - 2023)", # Título do gráfico
    # Adicionar a anotação com a fonte dos dados
    annotations = [
        dict(
            x=0, y=-0.2,  # Posição da fonte no layout (ajuste conforme necessário)
            xref="paper", yref="paper",
            text="Fonte: INMET, 2024",  # Texto da fonte
            showarrow=False,
            font=dict(size=10, color="gray")
        )
    ]
)

In [31]:
# Exportar o df para um arquivo csv

# est_brasilia_min_media_ano.rename(
#     columns={'NUM_MÊS':'num_mes',
#                 'MÊS':'nm_mes',
#                 'TEMPERATURA MINIMA, DIARIA(°C)':'temperatura_min_media_°C'
#              }, inplace=True)

# est_brasilia_min_media_ano['temperatura_min_media_°C'] = round(est_brasilia_min_media_ano['temperatura_min_media_°C'], 2)

# if not os.path.exists(caminho_arquivo('resultado')):
#     os.makedirs(caminho_arquivo('resultado'))
# est_brasilia_min_media_ano.to_csv(caminho_arquivo('resultado/est_brasilia_min_media_ano.csv'), index=False)

### SALVAR DADOS E GRÁFICOS EM UM EXCEL


In [32]:
est_brasilia_precipitacao_media_ano_sorted.rename(
    columns={'NUM_MÊS':'num_mes',
                'MÊS':'nm_mes',
                'PRECIPITACAO TOTAL, DIARIO(mm)':'precipitacao_mensal_media_mm'
             }, inplace=True)

est_brasilia_umidade_media_ano.rename(
    columns={'NUM_MÊS':'num_mes',
                'MÊS':'nm_mes',
                'UMIDADE RELATIVA DO AR, MEDIA DIARIA(%)':'umidade_relativa_ar_media_%'
             }, inplace=True)

est_brasilia_max_media_ano.rename(
    columns={'NUM_MÊS':'num_mes',
                'MÊS':'nm_mes',
                'TEMPERATURA MAXIMA, DIARIA(°C)':'temperatura_max_media_°C'
             }, inplace=True)

est_brasilia_min_media_ano.rename(
    columns={'NUM_MÊS':'num_mes',
                'MÊS':'nm_mes',
                'TEMPERATURA MINIMA, DIARIA(°C)':'temperatura_min_media_°C'
             }, inplace=True)


# Dicionário de Meses para abreviação
meses_dict = {
    'September': 'S',
    'October': 'O',
    'November': 'N',
    'December': 'D',
    'January': 'J/E',
    'February': 'F',
    'March': 'M',
    'April': 'A',
    'May': 'M',
    'June': 'J',
    'July': 'J',
    'August': 'A'
}

# Abreveação dos meses 
est_brasilia_precipitacao_media_ano_sorted['nm_mes'] = est_brasilia_precipitacao_media_ano_sorted['nm_mes'].replace(meses_dict)
est_brasilia_umidade_media_ano['nm_mes'] = est_brasilia_umidade_media_ano['nm_mes'].replace(meses_dict)
est_brasilia_max_media_ano['nm_mes'] = est_brasilia_max_media_ano['nm_mes'].replace(meses_dict)
est_brasilia_min_media_ano['nm_mes'] = est_brasilia_min_media_ano['nm_mes'].replace(meses_dict)


# Arrendondar os valores
est_brasilia_precipitacao_media_ano_sorted['precipitacao_mensal_media_mm'] = round(est_brasilia_precipitacao_media_ano_sorted['precipitacao_mensal_media_mm'], 2)
est_brasilia_umidade_media_ano['umidade_relativa_ar_media_%'] = round(est_brasilia_umidade_media_ano['umidade_relativa_ar_media_%'], 2)
est_brasilia_max_media_ano['temperatura_max_media_°C'] = round(est_brasilia_max_media_ano['temperatura_max_media_°C'], 2)
est_brasilia_min_media_ano['temperatura_min_media_°C'] = round(est_brasilia_min_media_ano['temperatura_min_media_°C'], 2)



In [33]:
# Configurações das abas da planilha excel com os gráficos
abas = {
    "precipitacao": {
        "dados": est_brasilia_precipitacao_media_ano_sorted,
        "name_add_series":'Média de Chuva (mm)',
        "name_set_title":'Média histórica de chuva mensal no Distrito Federal (1961 - 2023)',
        "name_set_x_axis":'Mês',
        "name_set_y_axis":'Média de Chuva (mm)',
        "min_y": 0,
        "max_y": 300
        },
    
    "umidade": {
        "dados":est_brasilia_umidade_media_ano,
        "name_add_series":'Média de Umidade (%)',
        "name_set_title":'Média histórica de umidade relativa do ar no Distrito Federal (1961 - 2023)',
        "name_set_x_axis":'Mês',
        "name_set_y_axis":'Média de Umidade (%)',
        "min_y": 0,
        "max_y": 85
        },
    "temperaturaMax":{
        "dados":est_brasilia_max_media_ano,
        "name_add_series":'Média de Temperatura Máxima (°C)',
        "name_set_title":'Média histórica de temperatura máxima no Distrito Federal (1961 - 2023)',
        "name_set_x_axis":'Mês',
        "name_set_y_axis":'Média de Temperatura Máxima (°C)',
        "min_y": 0,
        "max_y": 32        
    },
    "temperaturaMin":{
        "dados":est_brasilia_min_media_ano,
        "name_add_series":'Média de Temperatura Mínima (°C)',
        "name_set_title":'Média histórica de temperatura mínima no Distrito Federal (1961 - 2023)',
        "name_set_x_axis":'Mês',
        "name_set_y_axis":'Média de Temperatura Mínima (°C)',
        "min_y": 0,
        "max_y": 20
    }
}



In [34]:

# Criar um arquivo Excel com xlsxwriter
with pd.ExcelWriter('resultado/Graficos_elementos_climaticos_cap2.xlsx', engine='xlsxwriter') as writer:
    
    for sheet_nome, config in abas.items():
        
        df = config['dados'] # DataFrame com os dados
        nome_add_series = config['name_add_series'] # Nome da série no gráfico
        titulo = config['name_set_title'] # Título do gráfico
        nome_x_axis = config['name_set_x_axis'] # Nome do eixo X
        nome_y_axis = config['name_set_y_axis'] # Nome do eixo Y
        min = config['min_y'] # Valor mínimo do eixo Y
        max = config['max_y'] # Valor máximo do eixo Y
        
        # Escrever o DataFrame diretamente na planilha
        df.to_excel(writer, sheet_name=sheet_nome, index=False, startrow=1)
        
        # Acessar o workbook e worksheet
        workbook = writer.book
        worksheet = writer.sheets[sheet_nome]
        
        # Adicionar um gráfico de linhas
        chart = workbook.add_chart({'type': 'line'})
        
        # Configurar o gráfico com os dados de umidade e meses
        chart.add_series({
            'name': nome_add_series,
            'categories': [sheet_nome, 2, 1, 13, 1],  # Coluna dos meses no DataFrame
            'values':     [sheet_nome, 2, 2, 13, 2],  # Coluna dos valores de umidade
            'data_labels': {'value': True},        # Exibir valores nos pontos do gráfico
        })
        
        # Configurar título e eixos do gráfico
        chart.set_title({'name': titulo})
        chart.set_x_axis({'name': nome_x_axis})
        chart.set_y_axis({'name': nome_y_axis, 'min': min, 'max': max})
        
        # Configurar o tamanho do gráfico
        chart.set_size({'width': 860, 'height': 376})
        
        # Inserir o gráfico na planilha
        worksheet.insert_chart('E4', chart)

    # O arquivo `Graficos_elementos_climaticos_cap2.xlsx` será salvo no diretório resultado